In [1]:
from PIL import Image
import torch
import numpy as np

from utils import show_image_caption
from utils import get_device_map

/home/kjh/anaconda3/envs/my_env_NICE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
devices = [1, 5, 6, 7]
start_device = 'cuda:' + str(devices[0])

### Configs

In [3]:
# checkpoint = "Salesforce/blip2-opt-2.7b"
checkpoint = "Salesforce/blip2-flan-t5-xl"
result_file_path = '../results/coco_test_blip2.csv'
cache_dir = "/mnt/nas2/kjh/huggingface_cache"
dtype = torch.float16
batch_size = 32
num_workers = 8
max_new_tokens = 50

### Processor

In [4]:
from transformers import Blip2Processor

processor = Blip2Processor.from_pretrained(
    checkpoint,
    cache_dir=cache_dir,
)


### Model

In [5]:
from transformers import Blip2ForConditionalGeneration

device_map = get_device_map(checkpoint, devices)

model = Blip2ForConditionalGeneration.from_pretrained(
    checkpoint,
    cache_dir=cache_dir,
    torch_dtype=dtype,
    # device_map='auto',
    device_map=device_map
)

IndexError: list index out of range

### Inference Samples

In [ ]:
image = '../datasets/cvpr-nice-val/val/215268662.jpg'
raw_image = Image.open(image).convert('RGB')

inputs = processor(raw_image, return_tensors="pt").to(start_device, dtype)

In [ ]:
from utils import denormalize_image, plot_images

generated_ids = model.generate(**inputs, max_new_tokens=20)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

denormalized_image = denormalize_image(inputs['pixel_values'].cpu()[0], processor.image_processor.image_mean, processor.image_processor.image_std)
# show_image_caption(denormalized_image, [generated_text], show_fig=True, save_path='sample.png')
# show_image_caption(raw_image, [generated_text], show_fig=True)
plot_images(denormalized_image, generated_text)

NameError: name 'model' is not defined